Menggunakan model open source berbasis [Qwen 1.5-0.5B-Chat](https://huggingface.co/Qwen/Qwen1.5-0.5B-Chat) yang tadinya akan di fine tuning tetapi karena keterbatasan menjadi menggunanakan RAG

#Install & Import Library

In [ ]:
pip install -U pip

In [1]:
!pip install -U jedi torch torchvision torchaudio fastai langchain_community langchain transformers sentence-transformers chromadb accelerate bitsandbytes unstructured

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import warnings

# Mulai Melakukan RAG Step

## Load Model

In [7]:
# Mengabaikan peringatan yang tidak krusial
warnings.filterwarnings("ignore")

print("Tahap 1: Memuat Model LLM (Llama 3 8B Instruct)...")
model_id = "Qwen/Qwen1.5-0.5B-Chat"

# Muat tokenizer dan model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

# Buat pipeline untuk text generation menggunakan transformers
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
)

# Bungkus pipeline ini ke dalam format yang dimengerti LangChain
llm = HuggingFacePipeline(pipeline=pipe)
print("Model LLM berhasil dimuat.\n")

Tahap 1: Memuat Model LLM (Llama 3 8B Instruct)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

Device set to use cuda:0


Model LLM berhasil dimuat.



## Load & Chunking File(Knowledge Base)

In [ ]:
try:
    with open("/content/Markdown.md", "r", encoding="utf-8") as f:
        markdown_content = f.read()
    print(markdown_content)
except FileNotFoundError:
    print("Error: File /content/Markdown.md not found.")
except Exception as e:
    print(f"An error occurred: {e}")

## 1. Ad Hominem (Personal Attack)
### ID & Alias
- FALLACY_AD_HOMINEM; Personal Attack, Argumentum ad Hominem.

### Functional Category
- Fallacy of Relevance.

### Definition
- An attempt to refute or discredit an argument by attacking the character, motive, affiliation, or other personal attributes of the person making the argument, rather than attacking the substance of the argument itself.[17, 21]

### Reasoning Analysis (Why It's Fallacious)
- This fallacy occurs because a person's character, circumstances, or motives are logically irrelevant to the truth or falsehood of the claim they are making. An argument should be evaluated based on the strength of its premises and the validity of its reasoning (Logos), not on who is presenting it (Ethos).[2, 13] Attacking someone's credibility does not automatically invalidate the logic of their argument.

### Sub-Types
- **Abusive Ad Hominem:** A direct insult or disparagement of the opponent's character. Example: "You are an idiot, so eve

In [8]:
import re
import json

def chunk_markdown_by_level_three_header(file_path: str) -> list[dict]:
    """
    Chunks a Markdown file based on Level 3 headers (###) and captures
    the parent Level 2 header (##) as metadata.

    Args:
        file_path: The path to the .md file.

    Returns:
        A list of dictionaries, where each dictionary represents a chunk
        with its content and metadata.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return []

    # Split the document into parts based on Level 2 headers (##)
    # This helps in associating L3 headers with their L2 parent.
    parts = re.split(r'\n(?=## )', content)

    all_chunks = []

    # Process the part before the first ##, if it exists
    # This captures the introductory part of the document
    intro_content = parts[0].strip()
    # Find chunks in the intro part if any ### exist there
    initial_chunks = re.split(r'\n(?=### )', intro_content)
    # The first real chunk is everything before the first ###
    # Subsequent splits will contain the ### header
    if initial_chunks:
        # Get the main title (#) as the parent for the intro part
        main_title_match = re.search(r'^# (.+)', intro_content, re.MULTILINE)
        main_title = main_title_match.group(1).strip() if main_title_match else "Introduction"

        # Add the content before the first ### as its own chunk
        first_chunk_content = initial_chunks[0].strip()
        if first_chunk_content:
            all_chunks.append({
                "metadata": {
                    "parent_section_l2": main_title, # Using L1 title as parent here
                    "section_l3": main_title
                },
                "content": first_chunk_content
            })

        # Process remaining chunks within the intro part
        for chunk_text in initial_chunks[1:]:
             header_l3_match = re.search(r'^### (.+)', chunk_text, re.MULTILINE)
             header_l3 = header_l3_match.group(1).strip() if header_l3_match else "Unnamed Section"
             all_chunks.append({
                 "metadata": {
                     "parent_section_l2": main_title,
                     "section_l3": header_l3
                 },
                 "content": chunk_text.strip()
             })


    # Process parts starting from the first ##
    for part in parts[1:]:
        # Find the Level 2 header for the current part
        parent_header_match = re.search(r'^## (.+)', part)
        parent_header = parent_header_match.group(1).strip() if parent_header_match else "Unnamed Part"

        # Split the current part by Level 3 headers
        # The lookahead `(?=### )` keeps the delimiter (the header) in the next split
        chunks = re.split(r'\n(?=### )', part)

        # The first chunk in a part is the content under ## before the first ###
        # We need to handle this as a separate case.
        part_intro_content = chunks[0].strip()
        if part_intro_content:
            all_chunks.append({
                "metadata": {
                    "parent_section_l2": parent_header,
                    "section_l3": "Introduction" # Generic title for content under ##
                },
                "content": part_intro_content
            })

        # Process the rest of the chunks in the part
        for chunk_text in chunks[1:]:
            # Extract the Level 3 header for metadata
            header_l3_match = re.search(r'^### (.+)', chunk_text)
            header_l3 = header_l3_match.group(1).strip() if header_l3_match else "Unnamed Section"

            # Add the structured chunk to our list
            all_chunks.append({
                "metadata": {
                    "parent_section_l2": parent_header,
                    "section_l3": header_l3
                },
                "content": chunk_text.strip()
            })

    return all_chunks

file_name = '/content/Markdown.md' # Use the correct file path
chunks = chunk_markdown_by_level_three_header(file_name)

if chunks:
    print(f"✅ Berhasil membuat {len(chunks)} chunk dari file '{file_name}'.")
    print("\n--- Semua Hasil Chunking ---")

    for i, chunk in enumerate(chunks):
        print(f"--- Chunk {i+1} ---")
        print(json.dumps(chunk, indent=2))
        print("-" * 20)

✅ Berhasil membuat 22 chunk dari file '/content/Markdown.md'.

--- Semua Hasil Chunking ---
--- Chunk 1 ---
{
  "metadata": {
    "parent_section_l2": "Knowledge Base: Logic, Fallacies, and Chatbot Implementation",
    "section_l3": "Knowledge Base: Logic, Fallacies, and Chatbot Implementation"
  },
  "content": "# Knowledge Base: Logic, Fallacies, and Chatbot Implementation"
}
--------------------
--- Chunk 2 ---
{
  "metadata": {
    "parent_section_l2": "Part 1: Foundations of Logical Argumentation",
    "section_l3": "Introduction"
  },
  "content": "## Part 1: Foundations of Logical Argumentation\n\nThis section builds the theoretical foundation regarding the elements that constitute a strong and logical argument. This information serves as a fundamental principle for the AI."
}
--------------------
--- Chunk 3 ---
{
  "metadata": {
    "parent_section_l2": "Part 1: Foundations of Logical Argumentation",
    "section_l3": "1.1 Anatomy of an Argument: Premise, Inference, and Conclu

In [6]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import UnstructuredMarkdownLoader # Import the loader

# loader = UnstructuredMarkdownLoader("/content/Markdown.md")
# docs = loader.load()

# # Menggunakan RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,
#     chunk_overlap=50,
#     separators=["\n\n", "\n", " ", ""]
# )

# # Muat konten markdown dari file
# try:
#     with open("/content/Markdown.md", "r", encoding="utf-8") as f:
#         markdown_content = f.read()
# except FileNotFoundError:
#     print("Error: File /content/Markdown.md not found.")
#     markdown_content = "" # Atur konten kosong jika file tidak ditemukan

# if markdown_content:
#     chunks = text_splitter.split_text(markdown_content)
#     print(f"Dokumen berhasil dipecah menjadi {len(chunks)} bagian (chunks).\n")

#     # Tampilkan 10 chunk pertama
#     print("--- 10 Hasil Chunking Pertama ---")
#     for i, chunk in enumerate(chunks[:35]): # Limit the loop to the first 10 chunks
#         print(f"--- Chunk {i+1} ---")
#         print(chunk)
#         print("-" * 20) # Separator antar chunk
# else:
#     chunks = [] # Pastikan chunks kosong jika file tidak ditemukan
#     print("Tidak ada konten untuk dipecah.")

Dokumen berhasil dipecah menjadi 35 bagian (chunks).

--- 10 Hasil Chunking Pertama ---
--- Chunk 1 ---
# Knowledge Base: Logic, Fallacies, and Chatbot Implementation

## Part 1: Foundations of Logical Argumentation

This section builds the theoretical foundation regarding the elements that constitute a strong and logical argument. This information serves as a fundamental principle for the AI.

### 1.1 Anatomy of an Argument: Premise, Inference, and Conclusion
--------------------
--- Chunk 2 ---
Every argument is built from the same fundamental components. Understanding this anatomy is the first step in analyzing and constructing strong reasoning.
--------------------
--- Chunk 3 ---
- **Premise**: A statement put forward as a reason or evidence to support a claim. A premise serves as the foundation of an argument. It can be a verified fact, an assumption, or the conclusion of a previous argument.
- **Conclusion**: The statement that is drawn or deduced from the premises. It is the ma

## Embedding & Vector

In [10]:
embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_id)

texts_to_embed = [chunk["content"] for chunk in chunks]

vector_store = Chroma.from_texts(texts=texts_to_embed, embedding=embedding_model)
print("Vector store berhasil dibuat dengan ChromaDB.\n")

Vector store berhasil dibuat dengan ChromaDB.



## RAG Chain

In [13]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Buat template prompt untuk mengarahkan LLM
prompt_template = """
You are a logical and critical AI debate partner. Your mission is to respond to users' arguments with the aim of training their problem-solving and critical thinking skills[cite: 1].

You are debating on the topic: "[Debate Topic]".
Your position in this debate is: [AI Position: Pro/Con].
The User's position is: [User Position: Pro/Con].

Use the information from the context given below to build a strong and relevant counter-argument.
Context Information:
\"\"\"
{context}
\"\"\"

Your Task:
1.  Analyze the most recent argument from the user.
2.  Construct a counter-argument that logically challenges the user's claim.
3.  Defend your position as [AI Position: Pro/Con].
4.  Do not attack the user personally (avoid Ad Hominem)[cite: 6, 7].
5.  Present your argument in clear and structured Bahasa Indonesia.

Your counter-argument to this user's argument: {question}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Buat RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
    }
)
print("RAG Chain siap digunakan.\n")

RAG Chain siap digunakan.

